In [2]:
#Required packages
import pandas as pd

from fastai.collab import *

In [3]:
orders_df = pd.read_csv("./Data/olist_orders_dataset.csv")
order_items_df = pd.read_csv("./Data/olist_order_items_dataset.csv")
#customer_df = pd.read_csv("./Data/olist_customers_dataset.csv")
reviews_df = pd.read_csv("./Data/olist_order_reviews_dataset.csv")
product_df = pd.read_csv("./Data/olist_products_dataset.csv")

In [4]:
final_df = orders_df[['order_id', 'customer_id']].merge(order_items_df[['order_id', 'product_id']], on='order_id') \
                .merge(reviews_df[['order_id', 'review_score']], on='order_id') \
                .merge(product_df[['product_id', 'product_category_name']], left_on='product_id', right_on='product_id')
final_df.head()

,order_id,customer_id,product_id,review_score,product_category_name
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,87285b34884572647811a353c7ac498a,4,utilidades_domesticas
1,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,87285b34884572647811a353c7ac498a,4,utilidades_domesticas
2,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,87285b34884572647811a353c7ac498a,5,utilidades_domesticas
3,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,87285b34884572647811a353c7ac498a,3,utilidades_domesticas
4,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,595fac2a385ac33a80bd5114aec74eb8,4,perfumaria


In [5]:
# Create a DataLoaders object
dls = CollabDataLoaders.from_df(final_df, user_name='customer_id', item_name='product_id', rating_name='review_score', bs=64)

In [6]:
# Create a DataLoaders object
#dls = CollabDataLoaders.from_df(final_df, user_name='customer_id', item_name='product_id', rating_name='review_score', bs=64)

# Create and train the model
learn = collab_learner(dls, n_factors=50, y_range=(0, 5.5))
learn.fit_one_cycle(1, 5e-3)

epoch,train_loss,valid_loss,time
0,3.047470,3.035498,05:00


In [5]:
#learn.export()

In [3]:
# from fastai.tabular.all import load_learner

# # Replace 'path_to_model' with the actual path to your saved model
# path_to_model = 'item_based_v1.pkl'
# learn = load_learner(path_to_model)

In [8]:
# Generate recommendations for a given item
target_item = '87285b34884572647811a353c7ac498a'  # Replace with the actual item ID
item_idx = learn.dls.classes['product_id'].o2i[target_item]
item_factors = learn.model.i_weight.weight
similarity_scores = (item_factors @ item_factors[item_idx].T).squeeze()
recommended_indices = similarity_scores.argsort(descending=True)[:5]
recommended_items = [dls.classes['product_id'][i] for i in recommended_indices]

print(f"Recommended items for {target_item}: {recommended_items}")

Recommended items for 87285b34884572647811a353c7ac498a: ['3eef0cb94ba82de806bb30ab743c7655', '36f60d45225e60c7da4558b070ce4b60', '97f1396a5a1f7c07ba51784efdec44b8', '90df1d60e33ed1bdadec994d809f4bfd', '389d119b48cf3043d311335e499d9c6b']


In [10]:
from elasticsearch import Elasticsearch

# Initialize Elasticsearch client
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

# Delete the existing index if it exists
if es.indices.exists(index='item_recommendations'):
    es.indices.delete(index='item_recommendations')

# # Create the index with mappings
# index_settings = {
#     'mappings': {
#         'properties': {
#             'product_id': {'type': 'keyword'},
#             'recommendations': {
#                 'type': 'nested',
#                 'properties': {
#                     'rank': {'type': 'integer'},
#                     'recommended_item_id': {'type': 'keyword'}
#                 }
#             }
#         }
#     }
# }

# # Create the index
# es.indices.create(index='item_recommendations', body=index_settings)

C:\Users\chand\AppData\Local\Temp\ipykernel_30236\3657054374.py:7: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  if es.indices.exists(index='item_recommendations'):
C:\Users\chand\AppData\Local\Temp\ipykernel_30236\3657054374.py:8: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index='item_recommendations')


In [ ]:
# Iterate through all product IDs
for target_item in dls.classes['product_id']:
    item_idx = dls.classes['product_id'].o2i[target_item]
    item_factors = learn.model.i_weight.weight
    similarity_scores = (item_factors @ item_factors[item_idx].T).squeeze()
    recommended_indices = similarity_scores.argsort(descending=True)[:5]
    recommended_items = [dls.classes['product_id'][i] for i in recommended_indices]

    # Prepare the recommendations data
    recommendations = []
    for rank, item_id in enumerate(recommended_items, start=1):
        recommendations.append({
            'rank': rank,
            'recommended_item_id': item_id
        })

    # Save recommendations in Elasticsearch
    doc = {
        'product_id': target_item,
        'recommendations': recommendations
    }
    es.index(index='item_recommendations', id=target_item, body=doc)

C:\Users\chand\AppData\Local\Temp\ipykernel_27204\179936219.py:22: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  if not es.indices.exists(index='item_recommendations'):
C:\Users\chand\AppData\Local\Temp\ipykernel_27204\179936219.py:42: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index='item_recommendations', id=target_item, body={'recommendations': recommendations})
C:\Users\chand\AppData\Local\Temp\ipykernel_27204\179936219.py:42: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.e

: 

: 

In [11]:
# from elasticsearch import Elasticsearch

# # Initialize Elasticsearch client
# es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])  # Replace with your Elasticsearch host and port

# Create the index with mappings if it doesn't exist
index_settings = {
    'mappings': {
        'properties': {
            'product_id': {'type': 'keyword'},
            'recommendations': {
                'type': 'nested',
                'properties': {
                    'rank': {'type': 'integer'},
                    'recommended_item_id': {'type': 'keyword'}
                }
            }
        }
    }
}
if not es.indices.exists(index='item_recommendations'):
    es.indices.create(index='item_recommendations', body=index_settings)

# Iterate through all product IDs and save recommendations
for target_item in dls.classes['product_id']:
    item_idx = dls.classes['product_id'].o2i[target_item]
    item_factors = learn.model.i_weight.weight
    similarity_scores = (item_factors @ item_factors[item_idx].T).squeeze()
    recommended_indices = similarity_scores.argsort(descending=True)[:5]
    recommended_items = [dls.classes['product_id'][i] for i in recommended_indices]

    # Prepare the recommendations data
    recommendations = []
    for rank, item_id in enumerate(recommended_items, start=1):
        recommendations.append({
            'rank': rank,
            'recommended_item_id': item_id
        })

    # Save recommendations in Elasticsearch
    doc = {
        'product_id': target_item,
        'recommendations': recommendations
    }
    es.index(index='item_recommendations', id=target_item, body=doc)


C:\Users\chand\AppData\Local\Temp\ipykernel_30236\3566602912.py:21: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  if not es.indices.exists(index='item_recommendations'):
C:\Users\chand\AppData\Local\Temp\ipykernel_30236\3566602912.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index='item_recommendations', body=index_settings)
C:\Users\chand\AppData\Local\Temp\ipykernel_30236\3566602912.py:22: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.c

: 

: 